In [51]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium

print('Packages installed.')

Packages installed.


In [90]:
!pip install BeautifulSoup4

In [91]:
import numpy as np 
import pandas as pd 
import geocoder 
import requests 
from bs4 import BeautifulSoup 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

print('Libraries imported.')

Libraries imported.


In [92]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')


Data Collected.


In [93]:
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [64]:
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A,Not assigned,M2A,43.648690,-79.385440
1,M3A,North York,Parkwoods,43.752935,-79.335641
2,M4A,North York,Victoria Village,43.728102,-79.311890
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
4,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
5,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
6,M8A,Not assigned,M9A,43.648690,-79.385440
7,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
8,M2B,Not assigned,M3B,43.648690,-79.385440
9,M4B,East York,"Parkview Hill, Woodbine Gardens",43.707193,-79.311529


In [65]:
neighbors.shape

(135, 5)

In [66]:
VERSION = '20180605' 

neighborhood_name = neighbors.loc[0, 'Neighborhood']
neighborhood_latitude = neighbors.loc[0, 'Latitude'] 
neighborhood_longitude = neighbors.loc[0, 'Longitude'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

radius = 500
LIMIT = 100 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id=JGGBRN5XODTLZGJOMCSWIQMRH1JLGJKPSFR10XNB2R5U25GR&client_secret=KWRAMLK2HOJBQ2XLICLKXRU3M4HOCC1U2VG4Y4OPP5JF03QX&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

results = requests.get(url).json()
results


Latitude and longitude values of M2A
 are 43.648690000000045, -79.38543999999996.


{'meta': {'code': 200, 'requestId': '5ed4e751b9a389001baddc37'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [94]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [95]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Dip 'n Sip,Coffee Shop,43.678897,-79.297745


In [69]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [96]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [98]:
toronto_venues = getNearbyVenues(names=neighborhoods,
                                 latitudes=neighbors['Latitude'],
                                 longitudes=neighbors['Longitude'])

ValueError: Length mismatch: Expected axis has 0 elements, new values have 7 elements

In [72]:
print(toronto_venues.shape)
toronto_venues.head()

(675, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M2A,43.64869,-79.38544,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M2A,43.64869,-79.38544,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M2A,43.64869,-79.38544,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M2A,43.64869,-79.38544,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M2A,43.64869,-79.38544,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


In [73]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",5,5,5,5,5,5
"Bathurst Manor, Wilson Heights, Downsview North",5,5,5,5,5,5
Bayview Village,5,5,5,5,5,5
"Bedford Park, Lawrence Manor East",5,5,5,5,5,5
Berczy Park,5,5,5,5,5,5
"Birch Cliff, Cliffside West",5,5,5,5,5,5
"Brockton, Parkdale Village, Exhibition Place",5,5,5,5,5,5
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",5,5,5,5,5,5


In [74]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 5 uniques categories.


In [75]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Trail,Coffee Shop,Health Food Store,Neighborhood,Pub
0,1,0,0,M2A,0
1,0,0,1,M2A,0
2,0,0,0,M2A,1
3,0,0,0,M2A,0
4,0,1,0,M2A,0


In [76]:
toronto_onehot.shape

(675, 5)

In [77]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Trail,Coffee Shop,Health Food Store,Pub
0,Agincourt,0.2,0.2,0.2,0.2
1,"Alderwood, Long Branch",0.2,0.2,0.2,0.2
2,"Bathurst Manor, Wilson Heights, Downsview North",0.2,0.2,0.2,0.2
3,Bayview Village,0.2,0.2,0.2,0.2
4,"Bedford Park, Lawrence Manor East",0.2,0.2,0.2,0.2
5,Berczy Park,0.2,0.2,0.2,0.2
6,"Birch Cliff, Cliffside West",0.2,0.2,0.2,0.2
7,"Brockton, Parkdale Village, Exhibition Place",0.2,0.2,0.2,0.2
8,"Business reply mail Processing Centre, South C...",0.2,0.2,0.2,0.2
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.2,0.2,0.2,0.2


In [78]:
toronto_grouped.shape

(132, 5)

In [79]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Alderwood, Long Branch
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Bathurst Manor, Wilson Heights, Downsview North
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Bayview Village
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Bedford Park, Lawrence Manor East
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Berczy Park
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3         

               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----M1Y
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----M1Z
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----M2A
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----M3B
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----M3C
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----M3E
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Stor

               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Roselawn
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Rouge Hill, Port Union, Highland Creek
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Runnymede, Swansea
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Runnymede, The Junction North
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
----
               venue  freq
0              Trail   0.2
1        

In [80]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [81]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Agincourt,Pub,Health Food Store,Coffee Shop
1,"Alderwood, Long Branch",Pub,Health Food Store,Coffee Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Pub,Health Food Store,Coffee Shop
3,Bayview Village,Pub,Health Food Store,Coffee Shop
4,"Bedford Park, Lawrence Manor East",Pub,Health Food Store,Coffee Shop


In [82]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [83]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighbors

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M1A,Not assigned,M2A,43.648690,-79.385440,0,Pub,Health Food Store,Coffee Shop
1,M3A,North York,Parkwoods,43.752935,-79.335641,0,Pub,Health Food Store,Coffee Shop
2,M4A,North York,Victoria Village,43.728102,-79.311890,0,Pub,Health Food Store,Coffee Shop
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,0,Pub,Health Food Store,Coffee Shop
4,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211,0,Pub,Health Food Store,Coffee Shop


In [84]:
map_clusters = folium.Map(location=[43.67635739999999, -79.2930312], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [85]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Not assigned,0,Pub,Health Food Store,Coffee Shop
1,North York,0,Pub,Health Food Store,Coffee Shop
2,North York,0,Pub,Health Food Store,Coffee Shop
3,Downtown Toronto,0,Pub,Health Food Store,Coffee Shop
4,North York,0,Pub,Health Food Store,Coffee Shop
5,Downtown Toronto,0,Pub,Health Food Store,Coffee Shop
6,Not assigned,0,Pub,Health Food Store,Coffee Shop
7,Scarborough,0,Pub,Health Food Store,Coffee Shop
8,Not assigned,0,Pub,Health Food Store,Coffee Shop
9,East York,0,Pub,Health Food Store,Coffee Shop
